In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import json
import ast 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
'''
Notebook might not work as expected, because i was moving it, so paths might be incoherent. All datasets are now stored in 
    "main/datasets"

Making average word embeddings and storing them to .npy
'''

In [79]:
#Read and process
rows = []
with open('datasets/word_embedding_podcast_words.csv', 'r') as f:
    for line in f:
        #split to get word and vector
        parts = line.strip().split()
        word = parts[0]
        vector = [float(x) for x in parts[1:]]
        rows.append((word, vector))

wiki_words_df = pd.DataFrame(rows, columns=['word', 'vector'])



In [80]:
dataset = pd.read_csv('datasets/finalAndRatings.csv')

In [81]:
dataset

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,podcast_name,podtail_link,itunes_id,podcastindex_id,description,rss_link,language,categories,number_of_podcasts,rating,number_of_reviews,index_description
0,0,0,0,Dateline NBC,https://podtail.com/podcast/dateline-nbc/,1464919521,945141,"Current and classic episodes, featuring compel...",https://podcastfeeds.nbcnews.com/dateline-nbc,en,"News,Society,Culture,True Crime",469.0,4.4,34700.0,"[343, 2989, 2348, 2421, 5975, 506, 1256, 11779..."
1,1,2,2,SmartLess,https://podtail.com/podcast/smartless/,1521578868,150970,"SmartLess with Jason Bateman, Sean Hayes, & Wi...",https://feeds.simplecast.com/pvzhyDQn,en,"Comedy,Education,Society,Culture",184.0,4.7,47100.0,"[39, 6, 8405, 17979, 49, 16, 26, 6440, 172, 10..."
2,2,3,3,The Daily,https://podtail.com/podcast/the-daily/,1200361736,743229,This is what the news should sound like. The b...,http://rss.art19.com/the-daily,en,"News,Daily",1973.0,4.4,94800.0,"[19, 58, 548, 104, 926, 50, 3069, 1326, 84, 47..."
3,3,4,4,Stuff You Should Know,https://podtail.com/podcast/stuff-you-should-k...,278981407,827592,"If you've ever wanted to know about champagne,...",https://feeds.megaphone.fm/stuffyoushouldknow,en-US,"Society,Culture",2218.0,4.5,70500.0,"[46, 5810, 441, 1070, 161, 31, 9863, 7282, 456..."
4,4,5,5,Wait Wait... Don't Tell Me!,https://podtail.com/podcast/npr-programs-wait-...,121493804,55813,NPR's weekly news quiz hosted by Peter Sagal. ...,https://feeds.npr.org/344098539/podcast.xml,en,"Comedy,Leisure,Games",300.0,4.6,34900.0,"[113, 3845, 548, 10734, 3586, 15, 23, 5321, 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,14199,14215,14215,How To Buy a House,https://podtail.com/podcast/how-to-buy-a-house-1/,1705752628,6599180,"A simple ""How To"" Guide on how to buy a house ...",https://www.spreaker.com/show/5946861/episodes...,en,"Education,Business,Investing,Entrepreneurship,...",1.0,0.0,0.0,"[837, 105, 1614, 105, 1508, 422, 400, 1551, 19..."
14200,14200,14216,14216,Circular economy,https://podtail.com/podcast/circular-economy/,1542782236,3422103,Members of the interdisciplinary research netw...,https://anchor.fm/s/42509384/podcast/rss,en,"Science,Social",3.0,0.0,0.0,"[402, 9235, 206, 720, 10998, 1774, 7296, 671, ..."
14201,14201,14217,14217,Stylist in Your Back Pocket,https://podtail.com/podcast/stylist-in-your-ba...,1631362565,5580934,Wardrobe stylist Hanna Lee Style hosts a podca...,https://podcasts.helloaudio.fm/podcast/62d411d...,en,"Arts,Fashion,Beauty,Education,Self Improvement",9.0,0.0,0.0,"[12310, 16529, 690, 3347, 2662, 98, 91, 4382, ..."
14202,14202,14218,14218,Far Fetched Adventures,https://podtail.com/podcast/far-fetched-advent...,1450896459,339167,Every adventure is better with a dog (or any a...,https://farfetchedadventures.libsyn.com/rss,en,"Sports,Society,Culture,Personal,Journals",9.0,0.0,0.0,"[277, 5731, 220, 2087, 13, 60, 2003, 6, 542, 2..."


In [83]:
dataset['index_description'] = dataset['index_description'].apply(ast.literal_eval) #ast for '[..]' -> [..]


number_of_rows = len(dataset)
vector = 300  
average_vectors = np.zeros((number_of_rows, vector))
'''
For some reason i spend too much time debugging it.
I was trying to make a dictionary firstly, but saving it simply with json.dumps take toooo much time and ate all my 32 Gb of RAM
I didnt expect it to be that hard of a task for a poor computer
'''
for i, row in dataset.iterrows():
    indexes = row['index_description']
    if i%1000==0:
        print(f"{i} row is being processed") # feedback about processing stage
    vectors = wiki_words_df.loc[indexes, 'vector'].values
    avg_vector = np.mean(np.array(list(vectors)), axis=0)
    
    # Store the average vector in the array
    average_vectors[i, :] = avg_vector

0 row is being processed
1000 row is being processed
2000 row is being processed
3000 row is being processed
4000 row is being processed
5000 row is being processed
6000 row is being processed
7000 row is being processed
8000 row is being processed
9000 row is being processed
10000 row is being processed
11000 row is being processed
12000 row is being processed
13000 row is being processed
14000 row is being processed


In [84]:
average_vectors.shape

(14204, 300)

In [87]:
normalized_vectors = normalize(average_vectors)

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(normalized_vectors)


In [88]:
cosine_sim_matrix.shape

(14204, 14204)

In [89]:
np.save("datasets/wiki_avg_cos_sim.npy",cosine_sim_matrix)

In [92]:
# a function, that is a part of quering_functions in 'main', not important
def get_avg_wiki_emb_top(dataset,itunes_id,cosine_sim_matrix,top=10):
        try:
            code = int(itunes_id)
            row = dataset[dataset['itunes_id'] == code]
            if not row.empty:
                row = row.index[0]
            else:
                row = None
                return None
        except ValueError:
            row = dataset[dataset['podcast_name'].str.contains(itunes_id)]
            if not row.empty:
                row = row.index[0]
            else:
                row = None
                return None
            
        indexed_embeddings = set((i,cosine_sim_matrix[row][i]) for i in range(len(cosine_sim_matrix[row])))
        sorted_emb = sorted(indexed_embeddings,key=lambda item:item[1],reverse=True)
        sorted_emb = sorted_emb[:top]
        indices = [sorted_emb[i][0] for i  in range(len(sorted_emb))]
        return dataset.iloc[indices,:]

In [95]:
get_avg_wiki_emb_top(dataset,"Huberman",cosine_sim_matrix) #works good

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,podcast_name,podtail_link,itunes_id,podcastindex_id,description,rss_link,language,categories,number_of_podcasts,rating,number_of_reviews,index_description
11355,11355,11365,11365,Huberman Lab,https://podtail.com/podcast/huberman-lab/,1545953110,1365758,Huberman Lab discusses neuroscience: how our b...,https://hubermanlab.libsyn.com/rss,en,"Health,Fitness,Science",173.0,4.9,20400.0,"[3508, 5546, 11756, 105, 84, 1682, 40, 3478, 6..."
396,396,397,397,Brain Inspired,https://podtail.com/podcast/brain-inspired/,1428880766,786132,Neuroscience and artificial intelligence work ...,https://braininspired.co/feed/podcast,en-US,"Science,Natural,Technology,Education",99.0,4.8,122.0,"[11756, 5651, 1833, 79, 220, 478, 1682, 3399, ..."
8551,8551,8561,8561,Brain Health with Dr. Nissen,https://podtail.com/podcast/brain-health-with-...,1510757864,210792,Brain Health with Dr. Nissen brings you advanc...,https://anchor.fm/s/1d1af938/podcast/rss,en,"Health,Fitness,Medicine",34.0,4.9,14.0,"[1682, 466, 2883, 17591, 2128, 11756, 11903, 5..."
5140,5140,5147,5147,The Parasite Doc Podcast,https://podtail.com/podcast/the-parasite-doc-p...,1691966489,6428807,"Welcome to The Parasite Doc podcast, the show ...",https://app.kajabi.com/podcasts/2147514409/feed,en,"Health,Fitness,Medicine,Alternative,Mental",9.0,3.5,2.0,"[315, 11229, 194, 6, 8119, 6477, 244, 25273, 7..."
4313,4313,4320,4320,From Our Neurons to Yours,https://podtail.com/podcast/from-our-neurons-t...,1664298141,5952239,From Our Neurons to Yours is a show that criss...,https://feeds.buzzsprout.com/2112871.rss,en-us,"Science,Life,Health,Fitness,Mental",17.0,5.0,12.0,"[16, 84, 8195, 3475, 194, 6, 6585, 1502, 5526,..."
6839,6839,6849,6849,The Life Extension Podcast - Technology & Magi...,https://podtail.com/podcast/the-life-extension...,1582985283,4207495,The Life Extension Podcast discusses current e...,https://feed.podbean.com/lifeextension/feed.xml,en,"Health,Fitness,Medicine,Society,Culture,Science",23.0,5.0,1.0,"[172, 3224, 5546, 343, 1073, 2682, 3737, 626, ..."
1879,1879,1880,1880,Better with Dr. Stephanie,https://podtail.com/podcast/better-with-dr-ste...,1479074542,689167,"It isn't about being perfect, it's about being...",https://feeds.simplecast.com/PpiFYEvk,en,"Health,Fitness,Alternative,Nutrition,Medicine",337.0,4.8,438.0,"[12, 1435, 496, 31, 98, 1840, 12, 113, 31, 98,..."
13147,13147,13163,13163,Clinical Lab Chat,https://podtail.com/podcast/clinical-lab-chat/,1706446307,6583344,Step into the world of clinical labs with a po...,https://feeds.transistor.fm/clinical-lab-chat,en,"Science,Chemistry,Business",14.0,0.0,0.0,"[1520, 69, 244, 2985, 7478, 6, 24300, 1893, 69..."
13987,13987,14003,14003,Chemistry for the Future: Strange Substances a...,https://podtail.com/podcast/chemistry-for-the-...,732014313,318073,"It was here in Oxford, in the 1600s, that grea...",http://mediapub.it.ox.ac.uk/feeds/129362/audio...,en,Education,3.0,0.0,0.0,"[12, 14, 75, 23366, 6, 338, 4310, 89, 11, 4137..."
13984,13984,14000,14000,Chemistry for the Future: Strange Substances a...,https://podtail.com/podcast/chemistry-for-the-...,732019121,1087426,"It was here in Oxford, in the 1600s, that grea...",http://mediapub.it.ox.ac.uk/feeds/129362/video...,en,Education,3.0,0.0,0.0,"[12, 14, 75, 23366, 6, 338, 4310, 89, 11, 4137..."
